In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.model_selection import train_test_split
import cv2
from torch.optim import SGD
from torch.optim.lr_scheduler import LinearLR
from sklearn.preprocessing import OneHotEncoder

### GPU 설정

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

### 데이터 가져오기

In [3]:
dir_path = '/content/drive/MyDrive/고모부_머신러닝/dogncat'
path = []
dataset_type = []
label = []
for dir_name, _, file_names in os.walk(dir_path):
    for file_name in file_names:
        file_path = dir_name + '/' + file_name
        path.append(file_path)
    
        if '/training_set' in file_path:
            dataset_type.append('train')
        elif '/test_set' in file_path:
            dataset_type.append('test')
        else:
            dataset_type.append('N/A')

        if '/cats' in file_path:
            label.append('CAT')
        elif '/dogs' in file_path:
            label.append('DOG')
        else:
            label.append('N/A')

cnd_df = pd.DataFrame({'path' : path, 'type' : dataset_type, 'label' : label})
            

In [4]:
cnd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10032 entries, 0 to 10031
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    10032 non-null  object
 1   type    10032 non-null  object
 2   label   10032 non-null  object
dtypes: object(3)
memory usage: 235.2+ KB


In [5]:
cnd_df.head(10)

,path,type,label
0,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
1,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
2,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
3,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
4,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
5,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
6,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
7,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
8,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
9,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT


In [6]:
cnd_df = cnd_df[cnd_df['path'].str.contains('.jpg')]

In [7]:
cnd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10028 entries, 0 to 10031
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    10028 non-null  object
 1   type    10028 non-null  object
 2   label   10028 non-null  object
dtypes: object(3)
memory usage: 313.4+ KB


In [8]:
cnd_df.head()

,path,type,label
0,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
1,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
2,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
3,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
4,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT


In [9]:
# train, validation 구분
train_df, valid_df = train_test_split(cnd_df[cnd_df['type']=='train'], test_size = 0.25)

# onehotencoding
train_onehot = pd.get_dummies(train_df['label'])
valid_onehot = pd.get_dummies(valid_df['label'])

train_df = pd.concat([train_df, train_onehot], axis=1)
valid_df = pd.concat([valid_df, valid_onehot], axis=1)

train_df['label_idx'] = train_df.iloc[:, 3:].values.argmax(axis=1)
valid_df['label_idx'] = valid_df.iloc[:, 3:].values.argmax(axis=1)

In [10]:
train_df.iloc[:, 3:].values.argmax(axis=1)

array([0, 1, 1, ..., 0, 1, 0])

In [11]:
train_df

,path,type,label,CAT,DOG,label_idx
2749,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0,0
6180,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
8738,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
7957,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
3718,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0,0
...,...,...,...,...,...,...
8248,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
3234,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0,0
5076,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,CAT,1,0,0
10010,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1


In [12]:
print(len(train_df))
print(len(valid_df))

6003
2002


In [13]:
# sample용 데이터

sample_dog = train_df[train_df['label'] == 'DOG'].sample(30)
sample_cat = train_df[train_df['label'] == 'CAT'].sample(30)

sample_data = pd.concat([sample_dog, sample_cat])

In [14]:
sample_data

,path,type,label,CAT,DOG,label_idx
9912,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
7760,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
7264,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
8552,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
9322,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
9167,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
6215,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
6752,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
8207,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1
8855,/content/drive/MyDrive/고모부_머신러닝/dognc...,train,DOG,0,1,1


In [15]:
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 9912 to 3779
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   path       60 non-null     object
 1   type       60 non-null     object
 2   label      60 non-null     object
 3   CAT        60 non-null     uint8 
 4   DOG        60 non-null     uint8 
 5   label_idx  60 non-null     int64 
dtypes: int64(1), object(3), uint8(2)
memory usage: 2.5+ KB


### 커스텀 데이터 셋 만들기
- KL_div_loss를 사용하려면 onehot label과 인덱스 라벨 둘 다 필요하다.
- onehot label은 loss 계산용
- index label은 accuracy, recall, precision 계산용

In [16]:
class MyDataset2(Dataset):
    def __init__(self, df):
        super(MyDataset2, self).__init__()
        self.path = df['path'].values
        self.label = df['label_idx'].values # 클래스 idx가 들어있는 레이블 데이터. Accuracy, recall, precision 계산에 사용.
        self.label_onehot = df.iloc[:, 3:5].values # 클래스 idx를 onehot encoding한 레이블 데이터. kl_div loss 계산에 사용.
    
    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        image = cv2.cvtColor(cv2.imread(self.path[idx]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (244,244))
        image = np.asarray(image, dtype=np.float32).transpose(2,0,1)
        norm_image = (image - np.amin(image)) / (np.amax(image) - np.amin(image))

        label = np.asarray(self.label[idx], dtype=np.float32)
        label_onehot = np.asarray(self.label_onehot[idx], dtype=np.float32)

        return norm_image, label, label_onehot

In [17]:
train_dataset2 = MyDataset2(train_df)
valid_dataset2 = MyDataset2(valid_df)
sample_dataset2 = MyDataset2(sample_data)

### 데이터로더 만들기

In [18]:
train_loader2 = DataLoader(train_dataset2, batch_size=8, shuffle=True)
valid_loader2 = DataLoader(valid_dataset2, batch_size=8, shuffle=False)
sample_loader2 = DataLoader(sample_dataset2, batch_size=4, shuffle=True)

In [19]:
len(sample_loader2)

15

### 모델 만들기

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_size=3, padding='same')
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(5*244*244, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # print('input size :', x.shape)
        # print('input max :', torch.amax(x, dim=(1,2,3)))
        # print('input min :', torch.amin(x, dim=(1,2,3)))
        # print('input l2norm :', torch.linalg.vector_norm(x, dim=(1,2,3)))

        conv = self.conv1(x)
        # print('conv size :', conv.shape)
        # print('conv max :', torch.amax(conv, dim=(1,2,3)))
        # print('conv min :', torch.amin(conv, dim=(1,2,3)))
        # print('conv l2norm :', torch.linalg.vector_norm(conv, dim=(1,2,3)))

        conv_out = self.relu(conv)
        # print('conv_out size :', conv_out.shape)
        # print('conv_out max :', torch.amax(conv_out, dim=(1,2,3)))
        # print('conv_out min :', torch.amin(conv_out, dim=(1,2,3)))
        # print('conv_out l2norm :', torch.linalg.vector_norm(conv_out, dim=(1,2,3)))

        fc_input = conv_out.view(conv_out.size(0), -1)
        # print('fc_input size :', fc_input.shape)
        # print('fc_input max :', torch.amax(fc_input, dim=(1)))
        # print('fc_input min :', torch.amin(fc_input, dim=(1)))
        # print('fc_input l2norm :', torch.linalg.vector_norm(fc_input, dim=(1)))

        fc_logit = self.fc1(fc_input)
        # print('fc_logit size :', fc_logit.shape)
        # print('fc_logit max :', torch.amax(fc_logit, dim=(1)))
        # print('fc_logit min :', torch.amin(fc_logit, dim=(1)))
        # print('fc_logit l2norm :', torch.linalg.vector_norm(fc_logit, dim=(1)))
        # print()

        return fc_logit

In [21]:
model = CNN().to(device)
model

CNN(
  (conv1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (relu): ReLU()
  (fc1): Linear(in_features=297680, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

### loss function

In [22]:
loss_fn = nn.KLDivLoss(reduction='batchmean').to(device)

### optimizer, lr_scheduler

In [23]:
optimizer = SGD(model.parameters(), lr=0.001)
scheduler = LinearLR(optimizer)

### Confusion matrix, Accuracy, recall, precision

In [24]:
def calculate_conf_matrix(predicted:float, label:float, class_idx:float):
    TP_num = 0
    TN_num = 0
    FP_num = 0
    FN_num = 0
    for i in range(len(predicted)):
        if predicted[i] == class_idx and label[i] == class_idx:
            TP_num += 1

        elif predicted[i] == class_idx and label[i] != class_idx:
            FP_num += 1
            
        elif predicted[i] != class_idx and label[i] == class_idx:
            FN_num += 1

        elif predicted[i] != class_idx and label[i] != class_idx:
            TN_num += 1

    return TP_num, TN_num, FP_num, FN_num

def calculate_accuracy(conf_matrix_list):
    try:
        accuracy = (conf_matrix_list[0] + conf_matrix_list[1]) / (conf_matrix_list[0] + conf_matrix_list[1] + conf_matrix_list[2] + conf_matrix_list[3])
    except ZeroDivisionError:
        accuracy = 0
    return accuracy

def calculate_recall(conf_matrix_list):
    try: 
        recall = conf_matrix_list[0] / (conf_matrix_list[0] + conf_matrix_list[3])
    except ZeroDivisionError:
        recall = 0
    return recall

def calculate_precision(conf_matrix_list):
    try:
        precision = conf_matrix_list[0] / (conf_matrix_list[0] + conf_matrix_list[2])
    except ZeroDivisionError:
        precision = 0
    return precision

def calculate_acc_rec_pre(conf_matrix_dict : dict):
    for class_idx in conf_matrix_dict.keys():
        accuracy = calculate_accuracy(conf_matrix_dict[class_idx])
        recall = calculate_recall(conf_matrix_dict[class_idx])
        precision = calculate_precision(conf_matrix_dict[class_idx])

        print(f'class{class_idx} >>> accuracy : {accuracy}, recall : {recall}, precision : {precision}')

    

In [25]:
epoch = 30
running_loss = 0.
running_vloss = 0.
class_num = 2
accuracy = 0.
recall = 0.
precision = 0.

for i in range(epoch):
    print()
    print(f'========= Epoch{i+1} =========')
    print()
    # train
    model.train()
    for batch_idx, data in enumerate(sample_loader2):
        print(f'------ train batch{batch_idx + 1} ------')
        image_data, label_idx, label_onehot = data
        image_data = image_data.to(device)
        label_onehot = label_onehot.to(device)
        print('Label :', label_onehot)

        optimizer.zero_grad()
        tr_output= model(image_data)
        print('predicted :', tr_output)

        lsf = nn.LogSoftmax(dim=1)
        tr_output_lsf = lsf(tr_output)
        print('predicted(logsoftmax) :', tr_output_lsf)

        loss = loss_fn(tr_output_lsf, label_onehot) 
        print(f'train batch{batch_idx+1} loss(kl_div) :', loss)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

        if batch_idx + 1 == len(sample_loader2):
            avg_loss = running_loss / len(sample_loader2)
            print('Loss/train :', avg_loss)
            running_loss = 0.

    scheduler.step()

    print()

    # #evaluate
    model.eval()
    TP_TN_FP_FN = {}
    for batch_idx, val_data in enumerate(sample_loader2): # training data로 우선 evaluation해보기
        print(f'------ valid batch{batch_idx+1} ------')
        v_image, v_label, v_label_onehot = val_data
        v_image = v_image.to(device)
        v_label_onehot = v_label_onehot.to(device)
        val_output = model(v_image)
        val_output_lsf = lsf(val_output)
        val_output_idx = torch.argmax(val_output, dim=1)
        print(f'predicted : {val_output_idx}, actual : {v_label}')

        for i, class_idx in enumerate(range(class_num)):
                TP, TN, FP, FN = calculate_conf_matrix(val_output_idx, v_label, class_idx) # val_output_idx, valid_label 모두 벡터형태로 입력해야 한다.
                if batch_idx == 0:
                    TP_TN_FP_FN[class_idx] = np.array([TP, TN, FP, FN])
                else:
                    TP_TN_FP_FN[class_idx] += np.array([TP, TN, FP, FN])

        print(f'TP_TN_FP_FN : {TP_TN_FP_FN}')

        vloss = loss_fn(val_output_lsf, v_label_onehot)
        print(f'valid loss :', vloss)
        running_vloss += vloss.item()
        
        if batch_idx + 1 == len(sample_loader2):
            avg_vloss = running_vloss / len(sample_loader2)
            calculate_acc_rec_pre(TP_TN_FP_FN)
            print('Loss/valid :', avg_vloss)
            # print('Accuracy :', accuracy_score(valid_label.cpu().detach().numpy(), val_output_idx.cpu().detach().numpy()))
            # print('Recall :', recall_score(valid_label.cpu().detach().numpy(), val_output_idx.cpu().detach().numpy()))
            # print('Precision :', precision_score(valid_label.cpu().detach().numpy(), val_output_idx.cpu().detach().numpy()))
            running_vloss = 0.



========= Epoch1 =========

------ train batch1 ------
Label : tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])
predicted : tensor([[-0.0624,  0.3499],
        [-0.1404,  0.3477],
        [-0.1525,  0.0782],
        [-0.0243,  0.2058]], grad_fn=<AddmmBackward0>)
predicted(logsoftmax) : tensor([[-0.9204, -0.5081],
        [-0.9667, -0.4786],
        [-0.8151, -0.5844],
        [-0.8148, -0.5847]], grad_fn=<LogSoftmaxBackward0>)
train batch1 loss(kl_div) : tensor(0.6610, grad_fn=<DivBackward0>)
------ train batch2 ------
Label : tensor([[1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.]])
predicted : tensor([[-0.5007,  0.6802],
        [-0.4147,  0.5378],
        [-0.6206,  0.5645],
        [-0.5111,  0.6157]], grad_fn=<AddmmBackward0>)
predicted(logsoftmax) : tensor([[-1.4487, -0.2677],
        [-1.2787, -0.3263],
        [-1.4518, -0.2668],
        [-1.4075, -0.2807]], grad_fn=<LogSoftmaxBackward0>)
train batch2 loss(kl_div) : tensor(0.8623, grad_fn

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in long_scalars


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Label : tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.]])
predicted : tensor([[-0.4142,  0.4097],
        [ 1.6122, -1.5312],
        [-0.4573,  0.5539],
        [ 1.0529, -0.9675]], grad_fn=<AddmmBackward0>)
predicted(logsoftmax) : tensor([[-1.1876, -0.3638],
        [-0.0422, -3.1857],
        [-1.3215, -0.3103],
        [-0.1245, -2.1448]], grad_fn=<LogSoftmaxBackward0>)
train batch1 loss(kl_div) : tensor(0.2102, grad_fn=<DivBackward0>)
------ train batch2 ------
Label : tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])
predicted : tensor([[-1.0434,  1.0887],
        [ 0.9320, -0.8150],
        [ 0.2014, -0.1446],
        [-0.5161,  0.5048]], grad_fn=<AddmmBackward0>)
predicted(logsoftmax) : tensor([[-2.2441, -0.1121],
        [-0.1607, -1.9076],
        [-0.5351, -0.8810],
        [-1.3286, -0.3077]], grad_fn=<LogSoftmaxBackward0>)
train batch2 loss(kl_div) : tensor(0.2789, grad_fn=<DivBackward0>)
---